In [3]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from scipy.stats import uniform
import pandas as pd
import time

# Load dataset
X, y = load_iris(return_X_y=True)

# Define model
svm = SVC()

# Define parameter distributions for RandomizedSearchCV
param_dist = {
    'C': uniform(0.1, 10),
    'gamma': uniform(0.001, 1),
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

# -----------------------------
# 1️⃣ RANDOMIZED SEARCH
# -----------------------------
start_random = time.time()
random_search = RandomizedSearchCV(
    estimator=svm,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    random_state=42
)
random_search.fit(X, y)
end_random = time.time()

best_params_random = random_search.best_params_

# -----------------------------
# 2️⃣ GRID SEARCH (Refined)
# -----------------------------
param_grid_refined = {
    'C': [best_params_random['C'] * i for i in [0.5, 1, 1.5]],
    'gamma': [best_params_random['gamma'] * i for i in [0.5, 1, 1.5]],
    'kernel': [best_params_random['kernel']]
}

start_grid_refined = time.time()
grid_search_refined = GridSearchCV(
    estimator=svm,
    param_grid=param_grid_refined,
    cv=5,
    scoring='accuracy'
)
grid_search_refined.fit(X, y)
end_grid_refined = time.time()

# -----------------------------
# 3️⃣ PURE GRID SEARCH (Full search directly)
# -----------------------------
param_grid_full = {
    'C': [0.1, 0.5, 1, 5, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid']
}

start_grid_full = time.time()
grid_search_full = GridSearchCV(
    estimator=svm,
    param_grid=param_grid_full,
    cv=5,
    scoring='accuracy'
)
grid_search_full.fit(X, y)
end_grid_full = time.time()

# -----------------------------
# Combine results
# -----------------------------
results = {
    "Randomized Search": {
        "Best Params": best_params_random,
        "Best Accuracy": random_search.best_score_,
        "Time Taken (s)": round(end_random - start_random, 3)
    },
    "Grid Search (Refined)": {
        "Best Params": grid_search_refined.best_params_,
        "Best Accuracy": grid_search_refined.best_score_,
        "Time Taken (s)": round(end_grid_refined - start_grid_refined, 3)
    },
    "Grid Search (Full)": {
        "Best Params": grid_search_full.best_params_,
        "Best Accuracy": grid_search_full.best_score_,
        "Time Taken (s)": round(end_grid_full - start_grid_full, 3)
    }
}

# Display results as table
df_results = pd.DataFrame(results).T
print("\n🔍 Comparison: Randomized vs Grid Search (Refined & Full)\n")
print(df_results)



🔍 Comparison: Randomized vs Grid Search (Refined & Full)

                                                             Best Params  \
Randomized Search      {'C': 1.9340450985343383, 'gamma': 0.305242242...   
Grid Search (Refined)  {'C': 2.9010676478015074, 'gamma': 0.152621121...   
Grid Search (Full)        {'C': 0.5, 'gamma': 0.001, 'kernel': 'linear'}   

                      Best Accuracy Time Taken (s)  
Randomized Search              0.98          0.955  
Grid Search (Refined)      0.986667          0.278  
Grid Search (Full)         0.986667          3.798  
